In [1]:
from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
import time

# QuOCS and Qudi with Noise

## **What to do before**

* Update the QuOCS library
* Substitute the logic files in Qudi

To handle the noise in the physical system with QuOCS, we will use the **DCRABNoisyAlgorithm** class.
In particular, we will use the compensation drift functionality to reset the figure of merit at the beginning of every Super iteration to take into account any new drift term coming from the external noise sources.
Furthermore, a re-evaluation step method is added from the second iteration for every super-iteration. Indeed the measure is repeated n + 1 times, where n is the length of the probabilities list.

The changes to be made are in the algorithm settings and in the evaluation loop (See below). Both changes are marked in bold.

### Creation of the optimization dictionary

The optimization dictionary contains all the settings compulsory for the optimization algorithm in order to run a proper optimization. 

In [2]:
optimization_client_name = "test_dCRAB_Noisy_2_control_fields"

**CHANGE** :Now we will use the **dCRAB Noisy algorithm** which ensures **drift compensation** and handles **fluctuations** 

In [3]:
optimization_dictionary = {"optimization_client_name": optimization_client_name,
                           'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 
                           'opti_algorithm_class': 'DCrabNoisyAlgorithm', 
                          }

Number of iteration and super-iterations

In [4]:
# Total number of dCRAB superiteration
super_iteration_number = 5
# Maximum number of iteration per super-iteration
maximum_function_evaluations_number = 200

To activate the drift compensation and the re-evaluation steps add 

In [5]:
optimization_dictionary['algorithm_settings'] = { 'super_iteration_number': super_iteration_number, 
                                                 'maximum_function_evaluations_number': maximum_function_evaluations_number,
                                                "is_compensated_drift": True,
                                                "re_evaluation": {
                                                  "re_evaluation_steps": [0.3, 0.5, 0.51]}
}

Settings for the inner algorithm used by dCRAB

In [6]:
optimization_dictionary["dsm_settings"] = {'general_settings': {'dsm_name': 'nelder_mead', 'is_adaptive': False}, 
                                    'stopping_criteria': {'iterations_number': 100, 'xatol': 1e-10, 'frtol': 1e-10}}

### Times

We can define here how many times we need to use in the optimization

In [7]:
time_p = {'time_name': 'time_p', 'initial_value': 3.0}

In [8]:
optimization_dictionary["times"] = [time_p]

### Parameters

In [9]:
optimization_dictionary['parameters'] = []

### Pulses

In [10]:
pulse_amplitude = {'pulse_name': 'Amplitude', 
                   'upper_limit': 15.0, 
                   'lower_limit': 0.1, 
                   'bins_number': 101, 
                   'time_name': 'time_p', 
                   'amplitude_variation': 0.5, 
                   'basis': {'basis_name': 'Fourier', 
                             'basis_class': 'Fourier', 
                             'basis_module': 'quocslib.pulses.basis.Fourier', 
                             'basis_vector_number': 5, 
                             'random_super_parameter_distribution': 
                             {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                   'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                   'lower_limit': 0.1, 'upper_limit': 3.0}
                            }, 
                   'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                   'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: np.pi/3 + 0.0*t'}
                  }

In [11]:
pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 3.14, 
               'lower_limit': -3.14, 
               'bins_number': 101, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5, 
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': 5, 
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                          'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                          'lower_limit': 0.1, 'upper_limit': 3.0}}, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

In [12]:
optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [13]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}

Load the optimization algorithm into the optimization logic and display it into the GUI

In [14]:
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

Print the optimization dictionary also here

In [15]:
optimalcontrol.opti_comm_dict

{'optimization_dictionary': {'optimization_client_name': 'test_dCRAB_Noisy_2_control_fields', 'opti_algorithm_module': 'quocslib.optimalalgorithms.dCRABNoisyAlgorithm', 'opti_algorithm_class': 'DCrabNoisyAlgorithm', 'algorithm_settings': {'super_iteration_number': 5, 'maximum_function_evaluations_number': 200, 'is_compensated_drift': True, 're_evaluation': {'re_evaluation_steps': [0.3, 0.5, 0.51]}}, 'dsm_settings': {'general_settings': {'dsm_name': 'nelder_mead', 'is_adaptive': False}, 'stopping_criteria': {'iterations_number': 100, 'xatol': 1e-10, 'frtol': 1e-10}}, 'times': [{'time_name': 'time_p', 'initial_value': 3.0}], 'parameters': [], 'pulses': [{'pulse_name': 'Amplitude', 'upper_limit': 15.0, 'lower_limit': 0.1, 'bins_number': 101, 'time_name': 'time_p', 'amplitude_variation': 0.5, 'basis': {'basis_name': 'Fourier', 'basis_class': 'Fourier', 'basis_module': 'quocslib.pulses.basis.Fourier', 'basis_vector_number': 5, 'random_super_parameter_distribution': {'distribution_name': 'Un

**CHANGE** : We have to provide the standard deviation to the dCRAB algorithm. 

To do so we use the function:

``` fomlogic.update_fom(fom, std, status_code=0)```

after the measurement. **std** stays for the experimental standard deviation.

In [16]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": True}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    # wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            
    # If time exceeded, exit from the while loop and give a very high fom to the optimization algorithm
    # and status code -1, to interrupt the optimization smoothly
    if optimizationlogic.handle_exit_obj.is_user_running == False:
        print('Stopping the measurement!')
        # Update the pulse with the maximum number
        fomlogic.update_fom(10**10, status_code=-1)
        break
    #######################################################################################################
    # Get the Controls
    #######################################################################################################
    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    # Get the controls from the controls logic
    pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
    #######################################################################################################
    # Perform the measurement
    #######################################################################################################
    #
    #
    #######################################################################################################
    # Calculate the figure of merit and the standard deviation
    #######################################################################################################
    # calculate the Figure of Merit
    fom_dict = qubit.get_FoM(pulses, timegrids, parameters)
    # Extract the fom and std
    fom, std = fom_dict["FoM"], fom_dict["std"]
    #######################################################################################################
    # Return the figure of merit
    #######################################################################################################
    # Update the figure of merit and the standard deviation to the fom logic
    fomlogic.update_fom(fom, std, status_code=0)
    # update the last time the fom is calculated
    last_time_fom = time.time()
    #######################################################################################################
    # Optional part
    #######################################################################################################
    # Print the data just for debug purpose
    print("FoM: {fom}, Std: {std}, status_code: {status_code}".format(fom=fom, std=std, status_code=0))
print("Optimization finished")

Is noise? : True
FoM: 0.18560703087418995, status_code: 0
FoM: 0.198837012729543, status_code: 0
FoM: 0.013280362532424768, status_code: 0
FoM: 0.05962755868502375, status_code: 0
FoM: 0.04053639731494435, status_code: 0
FoM: 0.02283593636622111, status_code: 0
FoM: 0.09640406588467475, status_code: 0
FoM: 0.19672320312872627, status_code: 0
FoM: 0.33668560296872757, status_code: 0
FoM: 0.02941567335584987, status_code: 0
FoM: 0.06683340714509145, status_code: 0
FoM: 0.3797351459300421, status_code: 0
FoM: 0.13160930855996175, status_code: 0
FoM: 0.15136242360914784, status_code: 0
FoM: 0.29896288688178735, status_code: 0
FoM: 0.14204822634564654, status_code: 0
FoM: 0.3090784977364705, status_code: 0
FoM: 0.13056904241521758, status_code: 0
FoM: 0.5181745595406889, status_code: 0
FoM: 0.278601314021155, status_code: 0
FoM: 0.5805296939561075, status_code: 0
FoM: 0.4002889635572872, status_code: 0
FoM: 0.1449660516035291, status_code: 0
FoM: 0.4488201397415466, status_code: 0
FoM: 0.26

In [ ]:
# Access to the optimizer object to get info about the optimization
optimizer_obj = optimizationlogic.optimizer_obj

In [ ]:
optimizer_obj

In [ ]:
# Best fom with std
print("FoM: {fom} +- {std}".format(fom=optimizer_obj.best_fom, std=optimizer_obj.std) 

In [ ]:
# Best controls
pulses_list, time_grids_list, parameters_list = optimizer_obj.get_best_controls()
t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]